In [4]:
import numpy as np
from sklearn.datasets import fetch_california_housing

In [5]:
def rbf(x, c, s):
    return np.exp(-np.linalg.norm(x - c) ** 2 / (2 * s ** 2))


housing = fetch_california_housing()
X = housing.data
y = housing.target


training_size = int(0.6 * len(X))
X_train, X_test = X[:training_size], X[training_size:]
y_train, y_test = y[:training_size], y[training_size:]


def distance_based_centers(X_train, num_centers):
    centers = [X_train[np.random.choice(X.shape[0])]]
    for _ in range(1, num_centers):
        dist = np.array([np.min(np.linalg.norm(X_train - c, axis=1)) for c in centers])
        next_center = X_train[np.argmax(dist)]
        centers.append(next_center)
    return np.array(centers)

num_centers = 10
centers = distance_based_centers(X, num_centers)

def calculate_sigma(centers):
    d_max = np.max([np.linalg.norm(c1 - c2) for i, c1 in enumerate(centers) for c2 in centers[i+1:]])
    return d_max / np.sqrt(2 * len(centers))

sigma = calculate_sigma(centers)

R = np.zeros((X_train.shape[0], len(centers)))
for i in range(X_train.shape[0]):
    for j in range(len(centers)):
        R[i, j] = rbf(X_train[i], centers[j], sigma)

W = np.dot(np.linalg.pinv(R), y_train)

def rbf_network(X_train, centers, sigma, W):
    R = np.zeros((X_train.shape[0], len(centers)))
    for i in range(X_train.shape[0]):
        for j in range(len(centers)):
            R[i, j] = rbf(X_train[i], centers[j], sigma)
    return np.dot(R, W)

y_pred = rbf_network(X_test, centers, sigma, W)

def MSE(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

mse = MSE(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 2.737088899315867
